In [11]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='ajinda10_0051550143'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/keys/jiabhi_bastion'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/work/keys/jiabhi_sliver'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/work/keys/jiabhi_sliver.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()


In [12]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib
from fabrictestbed.slice_editor import Capacities

In [13]:
manager = fablib.get_default_fablib_manager()
try:
    print(f"{manager}")
except Exception as e:
    print(f"Exception: {e}")

AttributeError: 'FablibManager' object has no attribute 'project_id'

In [14]:
!cd /home/fabric/work/
!pwd

/home/fabric/work/jupyter-examples/fabric_examples


In [5]:
la_generator_dir = '/home/fabric/work/v4-la-tools/'
!$la_generator_dir/Search $la_generator_dir/LA/Jiabhi_Experiment_LA_Header.tsv $la_generator_dir/FD/Jiabhi_Experiment_Factors.tsv fixla OUT.tsv

Seed:	1649342811
0, 3
Adding t-way interactions
Went over 19 columns
Finished constructing CS Matrix
Original linear LA Score: 342
The matrix now has 1 rows
Score after finalized row: 174
The matrix now has 2 rows
Score after finalized row: 84
The matrix now has 3 rows
Score after finalized row: 42
The matrix now has 4 rows
Score after finalized row: 22
The matrix now has 5 rows
Score after finalized row: 8
The matrix now has 6 rows
Score after finalized row: 4
The matrix now has 7 rows
Score after finalized row: 0
Complete LA created with score: 0
Rows: 7

Other Stuff:
Columns in CSMatrix: 19


In [6]:
import csv
factors_dir = la_generator_dir+'FD/'
factors_tsv_file = open(factors_dir+'Jiabhi_Experiment_Factors.tsv')
read_tsv = csv.reader(factors_tsv_file, delimiter="\t")
parameters=[]
param_values=[]
next(read_tsv)
for row in read_tsv:
#     print(row)
    parameters.append(row[0])
    param_values.append(list(map(int, row[3:])))
print(parameters)
print(param_values)

['cores', 'ram', 'disk']
[[4, 8], [16, 32], [100, 500]]


In [7]:
la_dir = la_generator_dir+'LA/'
factors_tsv_file = open(factors_dir+'Jiabhi_Experiment_LA_Header.tsv')
read_tsv = csv.reader(factors_tsv_file, delimiter="\t")

FileNotFoundError: [Errno 2] No such file or directory: '/home/fabric/work/v4-la-tools/FD/Jiabhi_Experiment_LA_Header.tsv'

In [8]:
output_dir = '/home/fabric/work/jupyter-examples/fabric_examples/'
output_file = output_dir+'OUT.tsv'
output_tsv_file = open(output_file)
read_tsv = csv.reader(output_tsv_file, delimiter="\t")
next(read_tsv)
num_rows_cols = next(read_tsv)
print(num_rows_cols)
cols = int(num_rows_cols[1])+1
print(cols)
is_header = True
param_indexes = []
for row in read_tsv:
    if len(row)!=cols:
        continue
    print(row)
    if is_header:
        is_header = False
        continue
    param_indexes.append(list(map(int, row[:-1])))
print(param_indexes)

['7', '3']
4
['2', '2', '2', '']
['1', '0', '1', '']
['0', '1', '1', '']
['0', '0', '0', '']
['1', '1', '1', '']
['1', '0', '0', '']
['0', '0', '1', '']
['0', '1', '0', '']
[[1, 0, 1], [0, 1, 1], [0, 0, 0], [1, 1, 1], [1, 0, 0], [0, 0, 1], [0, 1, 0]]


In [9]:
try:
    params_config=[]
    for row in param_indexes:
        param_vs_value = {}
        for i in range(0, len(row)):
    #         print (i, row[i])
            param_name = parameters[i]
            param_vs_value[param_name] = param_values[i][row[i]]
    #         print(param_name, param_values[i][row[i]])
        params_config.append(param_vs_value)
#         print(param_vs_value)
    print(params_config)
except Exception as e:
    print(f"Exception: {e}")

[{'cores': 8, 'ram': 16, 'disk': 500}, {'cores': 4, 'ram': 32, 'disk': 500}, {'cores': 4, 'ram': 16, 'disk': 100}, {'cores': 8, 'ram': 32, 'disk': 500}, {'cores': 8, 'ram': 16, 'disk': 100}, {'cores': 4, 'ram': 16, 'disk': 500}, {'cores': 4, 'ram': 32, 'disk': 100}]


In [10]:
def create_slice(param_config, node1_name, node2_name, network1_name, network2_name, node1_nic_name, node2_nic_name):
    try:
        print(param_config)
        #Create Slice
        slice = fablib.new_slice(name=slice_name)

        # Node1
        node1 = slice.add_node(name=node1_name, site=site1)
        node1.set_capacities(**param_config)
        iface1 = node1.add_component(model='NIC_ConnectX_5', name=node1_nic_name).get_interfaces()[0]

        # Node2
        node2 = slice.add_node(name=node2_name, site=site2)
        node2.set_capacities(**param_config)
        iface2  = node2.add_component(model='NIC_ConnectX_5', name=node2_nic_name).get_interfaces()[0]

        # NetworkS
        net1 = slice.add_l3network(name=network1_name, interfaces=[iface1], type='IPv4')
        net2 = slice.add_l3network(name=network2_name, interfaces=[iface2], type='IPv4')

        #Submit Slice Request
        slice.submit()
    except Exception as e:
        print(f"Exception: {e}")

In [11]:
def get_slice(slice_name, node1_name, node2_name, network1_name, network2_name, node1_nic_name, node2_nic_name):
    try:
        slice = fablib.get_slice(name=slice_name)
        print(f"{slice}")
        return slice
    except Exception as e:
        print(f"Exception: {e}")

In [12]:
def print_slices(slice_name, node1_name, node2_name, network1_name, network2_name, node1_nic_name, node2_nic_name):
    try:
        slice = fablib.get_slice(name=slice_name)
        print(f"{slice.list_nodes()}")
    except Exception as e:
        print(f"Exception: {e}")

In [13]:
def get_node_available_ips(slice, node1_name, node2_name, network1_name, network2_name, node1_nic_name, node2_nic_name):
    try:
        network1 = slice.get_network(name=network1_name)
        network1_available_ips = network1.get_available_ips()
        print(f"{network1}")

        network2 = slice.get_network(name=network2_name)
        network2_available_ips =  network2.get_available_ips()
        print(f"{network2}")
        return network1, network2, network1_available_ips, network2_available_ips
    except Exception as e:
        print(f"Exception: {e}")

In [26]:
def configure_node1(slice, network1, network2, network1_available_ips, network2_available_ips, node1_name, node2_name, network1_name, network2_name, node1_nic_name, node2_nic_name):
    try:
        node1 = slice.get_node(name=node1_name) 
        print(node1)
        node1_iface = node1.get_interface(network_name=network1_name)  
        node1_iface.ip_addr_add(addr=network1_available_ips.pop(0), subnet=network1.get_subnet())

        node1.ip_route_add(subnet=network2.get_subnet(), gateway=network1.get_gateway())

        stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
        print (stdout)

        stdout, stderr = node1.execute(f'ip route list')
        print (stdout)
    except Exception as e:
        print(f"Exception: {e}")

In [28]:
def configure_node2_get_addr(slice, network1, network2, network1_available_ips, network2_available_ips, node1_name, node2_name, network1_name, network2_name, node1_nic_name, node2_nic_name):
    try:
        node2 = slice.get_node(name=node2_name)       
        print(node2)
        node2_iface = node2.get_interface(network_name=network2_name)
        node2_addr = network2_available_ips.pop(0)
        node2_iface.ip_addr_add(addr=node2_addr, subnet=network2.get_subnet())

        node2.ip_route_add(subnet=network1.get_subnet(), gateway=network2.get_gateway())

        stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
        print (stdout)

        stdout, stderr = node2.execute(f'ip route list')
        print (stdout)
        return node2_addr
    except Exception as e:
        print(f"Exception: {e}")

In [32]:
def execute_ping_test(node2_addr):
    try:
        node1 = slice.get_node(name=node1_name)        
        print(node2_addr)
        stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
        print (stdout)
        print (stderr)
        stdout = node1.execute(f'cat sender-ss.svg')
        print(stdout)

    except Exception as e:
        print(f"Exception: {e}")

In [33]:
itr = 0
for param in params_config:
    slice_name = 'jiabhi_test_slice_exp_'+str(itr)
    [site1,site2] = fablib.get_random_sites(count=2)
    print(f"Sites: {site1}, {site2}")

    node1_name = 'Node1'
    node2_name = 'Node2'

    network1_name='net1'
    network2_name='net2'

    node1_nic_name = 'nic1'
    node2_nic_name = 'nic2'
    
#     create_slice(param, node1_name, node2_name, network1_name, network2_name, node1_nic_name, node2_nic_name)
    slice = get_slice(slice_name, node1_name, node2_name, network1_name, network2_name, node1_nic_name, node2_nic_name)
    print_slices(slice_name, node1_name, node2_name, network1_name, network2_name, node1_nic_name, node2_nic_name)
    
    network1, network2, network1_available_ips, network2_available_ips = get_node_available_ips(slice, node1_name, node2_name, network1_name, network2_name, node1_nic_name, node2_nic_name)
    configure_node1(slice, network1, network2, network1_available_ips, network2_available_ips, node1_name, node2_name, network1_name, network2_name, node1_nic_name, node2_nic_name)
    node2_addr = configure_node2_get_addr(slice, network1, network2, network1_available_ips, network2_available_ips, node1_name, node2_name, network1_name, network2_name, node1_nic_name, node2_nic_name)
    execute_ping_test(node2_addr)
    itr+=1
    break

Sites: TACC, MICH
-----------  ------------------------------------
Slice Name   jiabhi_test_slice_exp_0
Slice ID     d22a80d6-74b7-488e-9bc5-1b2e6b4d82b8
Slice State  StableOK
Lease End    2022-04-08 14:55:06
-----------  ------------------------------------
ID                                    Name    Site    Host                          Cores    RAM    Disk  Image            Management IP                          State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  ---------------  -------------------------------------  -------  -------
4ee6a025-8b46-4593-900b-7d16e0b000d6  Node1   UTAH    utah-w4.fabric-testbed.net        8     32     500  default_rocky_8  2001:1948:417:7:f816:3eff:fe46:b774    Active
6ee89203-f0d4-450d-90c9-208a9d8f813d  Node2   WASH    wash-w3.fabric-testbed.net        8     32     500  default_rocky_8  2001:400:a100:3020:f816:3eff:fece:3c8  Active
-----------------  ---------------------------

In [ ]:
for i in range(0,5):
    try:
    #     print(slice_name)
        slice = fablib.get_slice(name='jiabhi_test_slice_exp_'+str(i))
        slice.delete()
    except Exception as e:
        print(f"Exception: {e}")

NameError: name 'node1' is not defined